In [8]:
#load variables from previous notebook
%store -r
print(model_package_arn_domain)
print(model_package_arn_instruct)
print(model_tar_gz_s3_domain)
print(model_tar_gz_s3_instruct)
print(test_input_path)

arn:aws:sagemaker:us-east-1:327216439222:model-package/gai-fine-tuned-domain-falcon-7b/3
arn:aws:sagemaker:us-east-1:327216439222:model-package/gai-fine-tuned-instruct-falcon-7b/3
s3://sagemaker-us-east-1-327216439222/huggingface-qlora-2023-07-30-23-29-12-2023-07-30-23-29-12-389/output/model.tar.gz
s3://sagemaker-us-east-1-327216439222/huggingface-qlora-2023-07-30-23-29-27-2023-07-30-23-29-27-188/output/model.tar.gz
s3://sagemaker-us-east-1-327216439222/model-fine-tuning/test/data_jsonlines_test.jsonl


In [1]:
#TODO remove
model_package_arn_domain = "arn:aws:sagemaker:us-east-1:327216439222:model-package/gai-fine-tuned-domain-falcon-7b/3"
model_package_arn_instruct = "arn:aws:sagemaker:us-east-1:327216439222:model-package/gai-fine-tuned-instruct-falcon-7b/3"
model_tar_gz_s3_domain = "s3://sagemaker-us-east-1-327216439222/huggingface-qlora-2023-07-30-23-29-12-2023-07-30-23-29-12-389/output/model.tar.gz"
model_tar_gz_s3_instruct = "s3://sagemaker-us-east-1-327216439222/huggingface-qlora-2023-07-30-23-29-27-2023-07-30-23-29-27-188/output/model.tar.gz"
model_name = "falcon-7b"
model_id = "tiiuae/falcon-7b"
test_input_path = "s3://sagemaker-us-east-1-327216439222/model-fine-tuning/test/data_jsonlines_test.jsonl"

In [9]:
!pip install huggingface_hub --upgrade --quiet
!pip install "transformers==4.30.2" "datasets[s3]==2.13.0" sagemaker --upgrade --quiet

If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it.



In [10]:
#required to work in local_mode on your notebook instance for development/debugging purpose
#!pip install 'sagemaker[local]' --upgrade --quiet
#!pip install docker-compose --quiet

In [11]:
import sagemaker
import boto3
import os

#uncomment to run in local mode
#from sagemaker import LocalSession
#sess = LocalSession()
#the below help setting up the container's root on the EBS volume of your instance.
#sess.config = {'local' : {'local_code' : True, 'container_root' : '/home/ec2-user/SageMaker/'}}
#if you're running local mode and run into out of space issues, consider running docker_scripts/prepare-docker.sh to set the docker root under /home/ec2-user/SageMaker

sess = sagemaker.Session()
region = sess.boto_region_name

#replace the below by a specific bucket if you need
sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
s3_client = boto3.client("s3")
s3_prefix = "model-fine-tuning"

#local notebook path
notebook_home = "/home/ec2-user/SageMaker/"

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {region}")

sagemaker role arn: arn:aws:iam::327216439222:role/Sagemaker
sagemaker bucket: sagemaker-us-east-1-327216439222
sagemaker session region: us-east-1


## Batch transform of the test dataset

In [ ]:
hyper_params = {"batch_size":4, "max_length":50, "top_k": 50, "top_p": 0.95, "do_sample": True}
hyper_params_dict = {"HYPER_PARAMS":str(hyper_params)}

batch_transformer = model.transformer(
    instance_count=1,
    instance_type=inference_instance_type,
    output_path=s3_output_data_path,
    assemble_with="Line",
    accept="text/csv",
    max_payload=1,
    env = hyper_params_dict
)

batch_transformer.transform(s3_input_data_path, content_type="application/jsonlines", split_type="Line")


In [33]:
from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=model_tar_gz_s3_domain,
   role=role, 
   transformers_version="4.28.1", 
   pytorch_version="2.0.0", 
   py_version="py310",
   model_server_workers=1,
)

In [34]:
#URL: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:2.0.0-transformers4.28.1-gpu-py310-cu118-ubuntu20.04
import time

transform_job_name = "hf-falcon7b" + time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())

hyper_params = {"do_sample": True, "top_p": 0.9, "temperature": 0.3, "max_new_tokens": 1024}
hyper_params_dict = {"HYPER_PARAMS":str(hyper_params)}

batch_job = huggingface_model.transformer(
    env = hyper_params_dict,
    instance_count=1,
    #instance_type='local_gpu',
    instance_type='ml.p3.16xlarge',
    strategy='SingleRecord')

INFO:sagemaker:Creating model with name: huggingface-pytorch-inference-2023-08-01-11-10-01-701


In [35]:
transform_job_name = "hf-falcon7b" + time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())

batch_job.transform(
    job_name=transform_job_name,
    data=test_input_path,
    content_type='application/json',
    split_type='Line')

INFO:sagemaker:Creating transform job with name: hf-falcon7b2023-08-01-11-10-04


........................................................................Collecting git+https://github.com/huggingface/peft.git@189a6b8e357ecda05ccde13999e4c35759596a67 (from -r /opt/ml/model/code/requirements.txt (line 3))
  Cloning https://github.com/huggingface/peft.git (to revision 189a6b8e357ecda05ccde13999e4c35759596a67) to /home/model-server/tmp/pip-req-build-1fmdezxq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /home/model-server/tmp/pip-req-build-1fmdezxq
  Running command git rev-parse -q --verify 'sha^189a6b8e357ecda05ccde13999e4c35759596a67'
  Running command git fetch -q https://github.com/huggingface/peft.git 189a6b8e357ecda05ccde13999e4c35759596a67
  Running command git checkout -q 189a6b8e357ecda05ccde13999e4c35759596a67
  Resolved https://github.com/huggingface/peft.git to commit 189a6b8e357ecda05ccde13999e4c35759596a67
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'


## Summarisation evaluation/metrics - ROUGE

ROUGE, or Recall-Orientpip install rouge-scoreed Understudy for Gisting Evaluation, is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation.

In the ROUGE paper, two flavors of ROUGE are described:


    sentence-level: Compute longest common subsequence (LCS) between two pieces of text. Newlines are ignored. This is called rougeL in this package.
    summary-level: Newlines in the text are interpreted as sentence boundaries, and the LCS is computed between each pair of reference and candidate sentences, and something called union-LCS is computed. This is called rougeLsum in this package. 

https://github.com/google-research/google-research/tree/master/rouge

In [30]:
!pip install rouge-score --quiet

### Example

In [31]:
examples_text = """Wine comedy up for six film gongs. Sideways, a wine-tasting comedy starring Paul Giamatti, is up for six Independent Spirit Awards, the art-house version of the Oscars.The awards are held on 26 February, the day before the Oscars. Spanish drama Maria Full of Grace, about a Colombian woman who becomes a drug courier, got five nominations. Controversial biopic Kinsey, starring Liam Neeson as sex researcher Alfred Kinsey, was one of four films to get four nominations. The awards, now in their 20th year, honour quirky low-budget films, all of which must have a degree of independent financing. Sideways is written and directed by Alexander Payne, who directed the 2002 hit About Schmidt, winning Jack Nicholson his 12th Academy Award nomination."These awards, for better or worse, mean everything," said Sideways producer Michael London, adding they were a "huge first step" toward getting recognition from other awards. Among the other films receiving four nominations apiece were Brother to Brother, a drama about a young gay black man forced to live on the streets, Robbing Peter and Primer. Primer, a $7,000 (£3,650) tale of discovery, won top prize at the Sundance film festival earlier this year. Walter Salles critically acclaimed The Motorcycle Diaries and the forthcoming thriller The Woodsman, starring Kevin Bacon, received three nominations each. Also in the running, with two nominations, are high school comedy Napoleon Dynamite, The Door in the Floor and Garden State - written, directed and starring Scrubs star Zach Braff alongside Natalie Portman. The awards were announced by actors Selma Blair and Dennis Quaid in Los Angeles on Tuesday."""
summary = """Sideways, a wine-tasting comedy starring Paul Giamatti, is up for six Independent Spirit Awards, the art-house version of the Oscars.Sideways is written and directed by Alexander Payne, who directed the 2002 hit About Schmidt, winning Jack Nicholson his 12th Academy Award nomination.Controversial biopic Kinsey, starring Liam Neeson as sex researcher Alfred Kinsey, was one of four films to get four nominations.Among the other films receiving four nominations apiece were Brother to Brother, a drama about a young gay black man forced to live on the streets, Robbing Peter and Primer.Also in the running, with two nominations, are high school comedy Napoleon Dynamite, The Door in the Floor and Garden State - written, directed and starring Scrubs star Zach Braff alongside Natalie Portman."""

In [32]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL', 'rougeLsum'], use_stemmer=True)
scores = scorer.score(examples_text, summary)
scores

INFO:absl:Using default tokenizer.


{'rouge1': Score(precision=1.0, recall=0.47388059701492535, fmeasure=0.6430379746835443),
 'rougeL': Score(precision=0.8503937007874016, recall=0.40298507462686567, fmeasure=0.5468354430379747),
 'rougeLsum': Score(precision=0.8503937007874016, recall=0.40298507462686567, fmeasure=0.5468354430379747)}

In [ ]:
os.path.join(notebook_home, data, "data_jsonlines_test.jsonl")
s3.download_file(test_input_path

In [ ]:
rouge1 = 0
rougeL = 0
rougeLsum = 0

with open(file, 'r') as file:
    lines = f.readlines()
    
for l in lines:
    scorer.score(examples_text, summary)

## Evaluation of the fine tuned model

We're using the LM Evaluation Harness framework to evaluate our fine tuned model

https://github.com/EleutherAI/lm-evaluation-harness

list of possible test/tasks to use:

https://github.com/EleutherAI/lm-evaluation-harness/blob/master/docs/task_table.md


### installation of lm-evaluation-harness

In [71]:
#installing the latest version of lm-evaluation-harness. note that the big-refactor is the new version branch that will probably become the main one soon.
#tested on revision 2820042d05e91c87852c82293f8973dc841c1a25 of the big-refactor branch
!git clone https://github.com/EleutherAI/lm-evaluation-harness && cd lm-evaluation-harness && git checkout big-refactor && pip install -e . --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 14128, done.
remote: Counting objects: 100% (2540/2540), done.
remote: Compressing objects: 100% (483/483), done.
remote: Total 14128 (delta 2264), reused 2158 (delta 2053), pack-reused 11588
Receiving objects: 100% (14128/14128), 19.21 MiB | 26.62 MiB/s, done.
Resolving deltas: 100% (9379/9379), done.
branch 'big-refactor' set up to track 'origin/big-refactor'.
Switched to a new branch 'big-refactor'


### Important

you need to modify lm-evaluation-harness/lm_eval/models/huggingface.py and add around line 60:

MODEL_FOR_CAUSAL_LM_MAPPING_NAMES["RefinedWebModel"] = "RWForCausalLM"

otherwise "self._config = transformers.AutoConfig.from_pretrained" set the model_type to "RefinedWebModel" which is not in the mapping list for MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.

as a result the auto_model_class selected  is transformers.AutoModelForSeq2SeqLM instead of transformers.AutoModelForCausalLM.

## Which tasks?

Not all tasks are available in the new refactored lm-evaluation-harness framework (you can always use the old version). see below for the WIP list:

https://github.com/EleutherAI/lm-evaluation-harness/tree/big-refactor/lm_eval/tasks

you'll find details under each folder for the tasks. the challenge is that there are 200+ tasks from the old frameworks so it is challenging to know which ones is the best of what you're trying to evaluate.

See below some resources that I found that can help:

https://super.gluebenchmark.com/tasks

https://gluebenchmark.com/tasks/

Notes from tasks that have already moved to the new framework:

- anli: Q&A
- ARC: Q&A science exam
- arithmetic
- hellaswag: sentence finishing
- lambada: next word prediction
- glue/qnli: Q&A https://rajpurkar.github.io/SQuAD-explorer/
- gsm8k: math problems
- headqa: health care complex reasoning
- hendrycks_ethics: ethic benchmark
- mathqa: math word problem solving
- openbookqa: Q&A??
- pile: general comprehension/Q&A across books, github repositories, webpages, chat logs, and medical, physics, math, computer science, and philosophy papers.
- piqa: commonsense Q&A, https://huggingface.co/datasets/piqa
- prost: contains 18,736 multiple-choice questions made from 14 manually curated templates, covering 10 physical reasoning concepts, https://paperswithcode.com/dataset/prost
- pubmedqa: biomedical Q&A https://github.com/pubmedqa/pubmedqa
- qa4mre: Q&A https://www.tensorflow.org/datasets/catalog/qa4mre
- race Q&A/comprehension https://huggingface.co/datasets/race
- sciq: crowdsourced science exam questions about Physics, Chemistry and Biology, https://huggingface.co/datasets/sciq
- webqs: WebQuestions is a benchmark for question answering.
- toxigen:  large-scale and machine-generated dataset of 274,186 toxic and benign statements about 13 minority groups.  https://paperswithcode.com/dataset/toxigen
- unscramble: Unscramble is a small battery of 5 “character manipulation” tasks. Each task involves giving the model a word distorted by some combination of scrambling, addition, or deletion of characters, and asking it to recover the original word.

### Useful Paths

In [90]:
lm_evaluation_git_dir = base_dir + "/lm-evaluation-harness"
lm_evaluation_git_dir

'/home/ec2-user/SageMaker/gai-finetuning/lm-evaluation-harness'

In [77]:
print(f"model_tuned_dir:{model_tuned_dir}")

model_tuned_dir:/home/ec2-user/SageMaker/models/falcon-7b-tuned/


In [112]:
output_file = "eval_tuned.txt"

### Select and launch the tasks

In [178]:
tasks = "hendrycks_ethics"

In [179]:
#to monitor GPU Memory usage on notebook instance, open a separate terminal and run: watch -n 0.4 nvidia-smi 
!cd evaluation_scripts && ./evaluation.sh hf $lm_evaluation_git_dir $model_tuned_dir $tasks > $output_file

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2023-07-24:04:08:41,819 INFO     [utils.py:148] Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-07-24:04:08:41,819 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2023-07-24:04:08:42,760 WARNING  [__init__.py:52] Failed to load config in
                                 /home/ec2-user/SageMaker/gai-finetuning/lm-evaluation-harness/lm_eval/tasks/hendrycks_ethics/utilitarianism_original.yaml
                                 Config will not be added to registry
                                 Error: argument of type 'NoneType' is not iterable
2023-07-24:04:08:42,975 INFO     [instantiator.py:21] Created a te

### Extract json from output

In [180]:
output_path = "./evaluation_scripts/" + output_file

#quick and dirty way to extract json in the midst of the text file
def extract_json(output_path):
    try:
        with open(output_path, 'r') as file:
                data_str = file.read()
                pos1 = int(data_str.find("{"))
                pos2 = int(data_str.rfind("}"))
                return json.loads(data_str[pos1:pos2+1])
    except Exception as e:
        print(f"cannot extract json from lm evaluation output, error: {e}")

In [235]:
json_obj = extract_json(output_path)

In [237]:
json_obj['results']

{'ethics_virtue': {'acc,none': 0.43537688442211053,
  'acc_stderr,none': 0.00703006143345469},
 'ethics_justice': {'acc,none': 0.5088757396449705,
  'acc_stderr,none': 0.009615647725764407},
 'ethics_deontology': {'acc,none': 0.5150166852057843,
  'acc_stderr,none': 0.008335364597109167},
 'ethics_cm': {'acc,none': 0.5917631917631918,
  'acc_stderr,none': 0.007886611421325077},
 'ethics_utilitarianism': {'acc,none': 0.5214226289517471,
  'acc_stderr,none': 0.007204999520618655}}

### Update model's metadata with results in model registry

In [233]:
#formatting the metrics as metadata to be stored in the model registry with the model
acc_none = 'acc,none'
acc_stderr = 'acc_stderr,none'

metadataProperties = {}
for key, value in json_obj['results'].items():
    metadataProperties[key + "_acc"] = str(json_obj['results'][key][acc_none])
    metadataProperties[key + "_acc_stderr"] = str(json_obj['results'][key][acc_stderr])

metadataProperties

{'ethics_virtue_acc': '0.43537688442211053',
 'ethics_virtue_acc_stderr': '0.00703006143345469',
 'ethics_justice_acc': '0.5088757396449705',
 'ethics_justice_acc_stderr': '0.009615647725764407',
 'ethics_deontology_acc': '0.5150166852057843',
 'ethics_deontology_acc_stderr': '0.008335364597109167',
 'ethics_cm_acc': '0.5917631917631918',
 'ethics_cm_acc_stderr': '0.007886611421325077',
 'ethics_utilitarianism_acc': '0.5214226289517471',
 'ethics_utilitarianism_acc_stderr': '0.007204999520618655'}

In [234]:
model_package_update_response = sm_client.update_model_package(ModelPackageArn=model_package_arn, CustomerMetadataProperties=metadataProperties)

## Next Steps 

You can deploy your fine-tuned model to a SageMaker endpoint and use it for inference. Check out the [Deploy Falcon 7B & 40B on Amazon SageMaker](https://www.philschmid.de/sagemaker-falcon-llm) and [Securely deploy LLMs inside VPCs with Hugging Face and Amazon SageMaker](https://www.philschmid.de/sagemaker-llm-vpc) for more details.